# Parameter Identification Technique (PIT) on static VCT with a linear VMM model

# Purpose
Is it possible to use a similar regression as used in [04.02_PIT_linear_VMM.ipynb](04.02_PIT_linear_VMM.ipynb) but with data from static Virtual Captive Tests (VCT)?

# Methodology
* Load static data from VCT
* Find the best parameter values in the linear model using OLS linear regression.

# Setup

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl
from vessel_manoeuvring_models.models import linear_vmm
import vessel_manoeuvring_models.linear_vmm_equations as eq
import vessel_manoeuvring_models.models.linear_vmm as model
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.plot import track_plot


## Load VCT data

In [ ]:
df_VCT_all = pd.read_csv('../data/external/vct.csv', index_col=0)
df_VCT_all.head()

In [ ]:
df_VCT = df_VCT_all.groupby(by=['model_name']).get_group('V2_5_MDL_modelScale')

In [ ]:
df_VCT.describe()

In [ ]:
df_VCT['test type'].unique()

# Subtract the resistance

In [ ]:
df_resistance = df_VCT.groupby(by='test type').get_group('resistance')

X = df_resistance[['u','fx']].copy()
X['u**2'] = X['u']**2
y = X.pop('fx')

model_resistance = sm.OLS(y,X)
results_resistance = model_resistance.fit()

X_pred = pd.DataFrame()
X_pred['u'] = np.linspace(X['u'].min(), X['u'].max(), 20)
X_pred['u**2'] = X_pred['u']**2
X_pred['fx'] = results_resistance.predict(X_pred)

fig,ax=plt.subplots()
df_resistance.plot(x='u', y='fx', style='.', ax=ax)
X_pred.plot(x='u', y='fx', style='--', ax=ax);


In [ ]:
df_VCT_0_resistance = df_VCT.copy()
df_VCT_0_resistance['u**2'] = df_VCT_0_resistance['u']**2
df_VCT_0_resistance['fx']-= results_resistance.predict(df_VCT_0_resistance[['u','u**2']])

## Load test

In [ ]:
df_runs = mdl.runs()

In [ ]:
#id=22773
#id=22616
#id=22774
id=22770


df, units, meta_data = mdl.load(id=id, dir_path='../data/processed/kalman')
df.index = df.index.total_seconds()
df = df.iloc[0:-100].copy()
df.index-=df.index[0]
df.sort_index(inplace=True)

In [ ]:
meta_data['rho']=1000
meta_data['mass'] = meta_data['Volume']*meta_data['rho']
meta_data.dropna()

In [ ]:
df.head()

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot
fig,ax=plt.subplots()
#fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

# Ship parameters

In [ ]:
T_ = (meta_data.TA + meta_data.TF)/2
L_ = meta_data.lpp
m_ = meta_data.mass
rho_ = meta_data.rho
B_ = meta_data.beam
CB_ = m_/(T_*B_*L_*rho_)
I_z_ = m_*meta_data.KZZ**2

ship_parameters = {
        'T' : T_,
        'L' : L_,
        'CB' :CB_,
        'B' : B_,
        'rho' : rho_,
        'x_G' : meta_data.lcg,  # motions are expressed at CG
        'm' : m_,
        'I_z': I_z_, 
    }

ps = prime_system.PrimeSystem(**ship_parameters)  # model

scale_factor = meta_data.scale_factor
ps_ship = prime_system.PrimeSystem(L=ship_parameters['L']*scale_factor, rho=meta_data['rho'])  # ship

# VCT to prime system

In [ ]:
interesting = [
    'u',
    'v',
    'r',
    'delta',
    'fx',
    'fy',
    'mz',
    'thrust',
]
df_VCT_prime = ps_ship.prime(df_VCT_0_resistance[interesting], U=df_VCT_0_resistance['V'])

In [ ]:
df_VCT_prime.describe()

# Linear Vessel Manoeuvring Model (VMM)
Implementing according to:
Matusiak, Jerzy. Dynamics of a Rigid Ship. Aalto University, 2017. https://aaltodoc.aalto.fi:443/handle/123456789/24408.

## N
The yaw is described by the following Ordinary Differential Equation (ODE):

In [ ]:
Math(vlatex(eq.N_eq))

The hydrodynamic moment part of this model is:

In [ ]:
Math(vlatex(eq.mz_eq))

In [ ]:
N_eq = eq.mz_eq.copy()
N_eq = N_eq.subs([
    #(x_G,0),  # Assuming or moving to CG=0
    #(I_z,1),  # Removing inertia
    #(eq.p.Nrdot,0),  # Removing added mass
    
    (u1d,0),
    (v1d,0),
    (r1d,0),

    
])  

#solution = sp.solve(N_eq,r1d)[0]
#N_eq = sp.Eq(r1d, solution*(I_z-eq.p.Nrdot))  # Putting r1d on the LHS

In [ ]:
#Math(vlatex(N_eq))

In [ ]:
diff_eq_N = regression.DiffEqToMatrix(ode=N_eq, label=N_lin, base_features=[delta,u,v,r])

If accelerations (dotted states) are dropped from this equation, the quasi-static force model is obtained:

In [ ]:
Math(vlatex(diff_eq_N.acceleration_equation))

This is a model that now only depend on steady velocities and rudder angle and will be the same thing as the forces/moment coming from the VCT calculations (CFD):

$ m_{z}^{VCT} = N_{lin} $

The hydrodynamic derivatives (the parameters in the matematical model) can be regressed by solving the following linear regression problem:

$ y = X \cdot \beta + \epsilon $

Where $y$ and $X$ and $\beta$ are:

In [ ]:
Math(vlatex(diff_eq_N.acceleration_equation_x))

In [ ]:
Math(vlatex(diff_eq_N.eq_y))

In [ ]:
diff_eq_N.eq_beta

In [ ]:
Math(vlatex(diff_eq_N.eq_X))

In [ ]:
diff_eq_N.X_lambda

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
def show_pred(X,y,results, label):
    
    display(results.summary())
    
    X_ = X.copy()
    X_['y'] = y
    X_.sort_values(by='y', inplace=True)
        
    y_ = X_.pop('y')
    
    y_pred = results.predict(X_)
    
    prstd, iv_l, iv_u = wls_prediction_std(results, exog=X_, alpha=0.05)
    #iv_l*=-1 
    #iv_u*=-1
    
    fig,ax=plt.subplots()
    #ax.plot(X_.index,y_, label='Numerical gradient from model test')
    #ax.plot(X_.index,y_pred, '--', label='OLS')
    
    ax.plot(y_,y_pred, '.')
    ax.plot([y_.min(),y_.max()], [y_.min(),y_.max()], 'r-')
    
    ax.set_ylabel(f'{label} (prediction)')    
    ax.set_xlabel(label)
    
    ax.fill_between(y_, y1=iv_l, y2=iv_u, zorder=-10, color='grey', alpha=0.5, label=r'5% confidence')
    ax.legend();

In [ ]:
X = diff_eq_N.calculate_features(data=df_VCT_prime)
y = diff_eq_N.calculate_label(y=df_VCT_prime['mz'])

model_N = sm.OLS(y,X)
results_N = model_N.fit()

show_pred(X=X,y=y,results=results_N, label=r'$N$')

## Y

In [ ]:
Math(vlatex(sp.solve(eq.Y_eom,Y_lin)))

In [ ]:
Math(vlatex(eq.Y_eom))

In [ ]:
Math(vlatex(eq.fy_eq))

In [ ]:
Y_eq = eq.fy_eq.copy()
Y_eq = Y_eq.subs([
    #(eq.p.Yvdot,1),  # Removing added mass
    #(x_G,0),  
    (u1d,0),
    (v1d,0),
    (r1d,0),
    
])  
#solution = sp.solve(Y_eq,v1d)[0]
#solution = solution*(-eq.p.Yvdot+m)
#solution = solution + U*m*r# (adding u*m*r to the measurement fy instead)  
#Y_eq = sp.simplify(sp.Eq(v1d, solution))  # Putting r1d on the LHS

In [ ]:
Math(vlatex(Y_eq))

In [ ]:
diff_eq_Y = regression.DiffEqToMatrix(ode=Y_eq, label=Y_lin, base_features=[delta,u,v,r])

In [ ]:
diff_eq_Y.eq_beta

In [ ]:
diff_eq_Y.acceleration_equation_x

In [ ]:
X = diff_eq_Y.calculate_features(data=df_VCT_prime, simplify_names=True)
y = diff_eq_Y.calculate_label(y=df_VCT_prime['fy'])
#y+=df['u']*df['r']  # adding centrifugal force

model_Y = sm.OLS(y,X)
results_Y = model_Y.fit()

show_pred(X=X,y=y,results=results_Y, label=r'$Y$')

In [ ]:
X.head()

## X

In [ ]:
X_eq = eq.fx_eq.copy()
X_eq = X_eq.subs([
    #(eq.p.Xudot,1),  # Removing added mass
    (u1d,0),
    (v1d,0),
    (r1d,0),
    #(m,0),  # mass
    
])  

#solution = sp.solve(X_eq,u1d)[0]
#X_eq = sp.Eq(u1d, solution*(-eq.p.Xudot+m))  # Putting r1d on the LHS

In [ ]:
Math(vlatex(X_eq))

In [ ]:
diff_eq_X = regression.DiffEqToMatrix(ode=X_eq, label=X_lin, base_features=[delta,u,v,r])

In [ ]:
X = diff_eq_X.calculate_features(data=df_VCT_prime)
y = diff_eq_X.calculate_label(y=df_VCT_prime['fx'])

model_X = sm.OLS(y,X)
results_X = model_X.fit()

show_pred(X=X,y=y,results=results_X, label=r'$X$')

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

# Brix parameters

In [ ]:
def calculate_prime(row, ship_parameters):
    return run(function=row['brix_lambda'], inputs=ship_parameters)

mask = df_parameters['brix_lambda'].notnull()
df_parameters.loc[mask,'brix_prime'] = df_parameters.loc[mask].apply(calculate_prime, ship_parameters=ship_parameters, axis=1)
df_parameters.loc['Ydelta','brix_prime'] = 0.005  # Just guessing
df_parameters.loc['Ndelta','brix_prime'] = -df_parameters.loc['Ydelta','brix_prime']/2  # Just guessing
df_parameters['brix_prime'].fillna(0, inplace=True)

## Add the regressed parameters
Hydrodynamic derivatives that depend on acceleration cannot be obtained from the VCT regression. They are however essential if a time simulation should be conducted. These values have then been taken from Brix semi empirical formulas for the simulations below.

In [ ]:
df_parameters_all = df_parameters.copy()
for other in [results_summary_X, results_summary_Y, results_summary_N]:
    df_parameters_all = df_parameters_all.combine_first(other)

df_parameters_all.rename(columns={'coeff':'regressed'}, inplace=True)
df_parameters_all.drop(columns=['brix_lambda'], inplace=True)

df_parameters_all['prime'] = df_parameters_all['regressed'].combine_first(df_parameters_all['brix_prime'])  # prefer regressed

In [ ]:
df_parameters_all

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,5)
df_parameters_all.plot.bar(y=['brix_prime','regressed'], ax=ax);

It seems that the regressed hydrodynamic derivatives are reasonable similar to the corresponding predicted values with semi-empirical formulas from Brix.

# Simulation

In [ ]:
df_ = df.copy()
t = df_.index
control = df_[['delta']]
#control = {'delta':0,}

df_0 = df_.iloc[0:100].median(axis=0)
y0 = {
    'u' : df_0['u'], 
    'v' : df_0['v'],
    'r' : df_0['r'],
    'x0' : df_0['x0'],
    'y0' : df_0['y0'],
    'psi' : df_0['psi'],
    }

#solution = model.simulate(y0=y0, t=t, df_parameters=df_parameters, df_ship_parameters=df_ship_parameters, control=control, rtol=1e-3, atol=1e-3)
solution = model.simulate(y0=y0, t=t, df_parameters=df_parameters_all, ship_parameters=ship_parameters, control=control)

columns = list(y0.keys())
df_result_prime = pd.DataFrame(data=solution.y.T, columns=columns)
df_result_prime.index=t[0:len(df_result_prime)]

#df_result_prime['U'] = np.sqrt(df_result_prime['u']**2 + df_result_prime['v']**2)

df_result_prime.plot(y='u')
df_result_prime.plot(y='v')
df_result_prime.plot(y='r')


In [ ]:
U_ = np.sqrt(df_0['u']**2 + df_0['v']**2)
df_result = ps.unprime(values=df_result_prime, U=U_)
df_result['beta'] = -np.arctan2(df_result['v'],df_result['u'])

In [ ]:
fig,ax=plt.subplots()
track_plot(df=df, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='model test')
track_plot(df=df_result, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='simulation', color='green')
ax.legend()


for key in df_result:
    fig,ax = plt.subplots()
    df.plot(y=key, label='model test', ax=ax)
    df_result.plot(y=key, label='simulation', ax=ax)
    ax.set_ylabel(key)

